--------------------------------------------------------------------------------
Travel time calculation using ModPathWell class, combined with 'omp removal'
--------------------------------------------------------------------------------

Example to calculate the travel time distribution per geological layer for a 
.. phreatic scheme, and to calculate the removal of organic micropollutants (OMP). 
.. For an extensive description of the steps, see the basic example [AnalyticalWell & OMP removal](https://github.com/KWR-Water/sutra2/blob/main/research/Tutorial_AW_omp_removal.ipynb).

________________________________________

Import packages
-----------------

First we import the necessary python packages

In [5]:
import pandas as pd
from pathlib import Path
from set_cwd_to_project_root import project_root

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from pandas import read_csv
from pandas import read_excel
import math
from scipy.special import kn as besselk
import sutra2.Analytical_Well as AW
import sutra2.ModPath_Well as mpw
import sutra2.Transport_Removal as TR

import warnings 
warnings.filterwarnings(action= 'ignore')

# get directory of this file
path = Path(project_root)
# Create research files dir
researchfiles_dir = os.path.join(path,"test_files")
if not os.path.exists(researchfiles_dir):
    os.makedirs(researchfiles_dir)

%load_ext autoreload
%autoreload 2

NameError: name '__file__' is not defined

Step 1: Define the HydroChemicalSchematisation
-----------------------------------------------
The first step is to define the hydrogeochemistry of the system using the HydroChemicalSchematisation class.

In [ ]:
# Lets start with a simple example defining a HydroChemicalSchematisation object for a phreatic aquifer:
phreatic_schematisation = AW.HydroChemicalSchematisation(schematisation_type='phreatic',
                                                    computation_method = 'modpath',
                                                    well_discharge=-7500, #m3/day
                                                    recharge_rate=0.0008, #m/day
                                                    thickness_vadose_zone_at_boundary=5, #m
                                                    thickness_shallow_aquifer=10,  #m
                                                    thickness_target_aquifer=40, #m
                                                    hor_permeability_target_aquifer=35, #m/day
                                                    redox_vadose_zone='anoxic',
                                                    redox_shallow_aquifer='anoxic',
                                                    redox_target_aquifer='deeply_anoxic',
                                                    pH_target_aquifer=7.,
                                                    temp_water=11.,
                                                    diffuse_input_concentration = 100, #ug/L
                                                    )

Then, we create a ModpathWell object for the HydroChemicalSchematisation object that we just made.
The ModpathWell object requires a dictionary of the subsurface schematisation and a set of boundary conditions the numerical model has to abide by in calculating flow velocity and direction of flow.

In [ ]:
phreatic_schematisation.make_dictionary()

Step 2: Run the ModpathWell class
=====================================
Next we create an ModpathWell object for the HydroChemicalSchematisation object we just made.
The data files will be stored in location workspace using a given modelname.

In [6]:
modpath_phrea = mpw.ModPathWell(phreatic_schematisation,
                            workspace = os.path.join(path,"omp_removal"),
                            modelname = "phreatic")

NameError: name 'path' is not defined